In [0]:
# Mount ADLS Gen2
# Required each time the cluster is restarted which should be only on the first notebook as they run in order

tiers = ["bronze", "silver", "gold"]
adls_paths = {tier: f"abfss://{tier}@deprojectearthquake.dfs.core.windows.net/" for tier in tiers}

# Accessing paths
bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"] 

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

[]

In [0]:
import requests
import json
from datetime import date, timedelta

In [0]:
start_date = date.today() - timedelta(1)
end_date = date.today()

In [0]:
start_date, end_date

(datetime.date(2025, 8, 25), datetime.date(2025, 8, 26))

In [0]:
# Construct the API URL with start and end dates provided by Data Factory, formatted for geojson output.
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

In [0]:
url

'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2025-08-25&endtime=2025-08-26'

In [0]:
try:
    # Make the GET request to fetch data
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    data = response.json().get('features', [])

    if not data:
        print("No data returned for the specified date range.")
    else:
        # Specify the ADLS path
        file_path = f"{bronze_adls}/{start_date}_earthquake_data.json"

        # Save the JSON data
        json_data = json.dumps(data, indent=4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f"Data successfully saved to {file_path}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")


Wrote 302087 bytes.
Data successfully saved to abfss://bronze@deprojectearthquake.dfs.core.windows.net//2025-08-25_earthquake_data.json


In [0]:
data[0]

{'type': 'Feature',
 'properties': {'mag': 0.64,
  'place': '67 km WNW of Tyonek, Alaska',
  'time': 1756165643860,
  'updated': 1756182644700,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/av93734213',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=av93734213&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 6,
  'net': 'av',
  'code': '93734213',
  'ids': ',av93734213,',
  'sources': ',av,',
  'types': ',origin,phase-data,',
  'nst': 9,
  'dmin': 0.03733,
  'rms': 0.06,
  'gap': 133,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 0.6 - 67 km WNW of Tyonek, Alaska'},
 'geometry': {'type': 'Point', 'coordinates': [-152.295, 61.2895, -0.12]},
 'id': 'av93734213'}

In [0]:
# # Define your variables
# output_data = {
#     "start_date": start_date.isoformat(),
#     "end_date": end_date.isoformat(),
#     "bronze_adls": bronze_adls,
#     "silver_adls": silver_adls,
#     "gold_adls": gold_adls
# }

# # Return the dictionary directly
# dbutils.jobs.taskValues.set(key = "bronze_output", value = output_data)